In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from collections import Counter

# pd.set_option('display.max_colwidth', None)

In [4]:
df = pd.read_csv('steam_games.csv')

def wrapErrorDelete(path):
    try:
        os.remove(path)
    except:
        pass
        
def p(x):
    print(x)

In [5]:
df = df[df['name'].notna()]
df.drop(["discount_price", "publisher", "developer", "release_date", "desc_snippet", "types", "achievements", "recent_reviews", "mature_content"], axis=1, inplace=True)

In [6]:
for row in df.iterrows():
    if pd.isna(row[1][3]) and pd.isna(row[1][6]):
        print("deleting: ", row[1][1])
        df.drop(row[0], inplace=True)    

deleting:  Ghost In The Shell: Stand Alone Complex 2nd Gig
deleting:  Ghost In The Shell: Stand Alone Complex Season 1
deleting:  Gintama Season 4
deleting:  CRYENGINE - Sample Assets
deleting:  Worms Reloaded: The "Pre-order Forts and Hats" DLC Pack
deleting:  Krater - Soundtrack
deleting:  Dustforce Soundtrack
deleting:  The Incredible Adventures of Van Helsing II - OST
deleting:  D4: SWERY's Choice Costume Set -4 Cups of Coffee-
deleting:  D4: SWERY's Choice Costume Set -4 Bottles of Tequila-
deleting:  Kemono Friends Season 1
deleting:  New Game Season 1
deleting:  Showa Genroku Rakugo Shinju Season 1
deleting:  Con Man Season 1
deleting:  DayZ Tools
deleting:  Forge of Gods: Ultimate Titan Bundle
deleting:  Killing Floor "London's Finest" Character Pack
deleting:  Dungeon Defenders Capture the Flag Pre-Alpha Pass (Free DLC)
deleting:  Digital Sound Factory: Session Drummer 3 Expansions
deleting:  Mob Psycho 100 Season 1
deleting:  Re:ZERO -Starting Life in Another World- Season 1


In [16]:
duplicated_items = df[df["name"].duplicated()]["name"].values.tolist()
games = {}
print(df.shape)
for row in df.iterrows():
    if row[1][1] in duplicated_items:
        if row[1][1] in games:
            games[row[1][1]].append(row[1].isna().sum())
        else:
            games[row[1][1]] = [row[1].isna().sum()]
for item in games:
    games[item].sort()
    count = 0
    for row in df.iterrows():
        if row[1][1] == item and (row[1].isnull().sum() == games[item][0]):
            df.drop(row[0], inplace=True)
            count += 1
            print("deleting: ", row[1][1])
        
        if count + 1 == len(games[item]):
            break
print(df.shape)
df.to_csv("steam_games_p1.csv", index=False)

(40328, 13)
deleting:  Game + Soundtrack
deleting:  Game + Soundtrack
(40326, 13)


In [ ]:
df

In [14]:
df = pd.read_csv('steam_games_p1.csv')
all_review_cols = ["all_review_sentiment", "all_review_sentiment_num", "all_review_num"]
temp_df = []
idx = 0
for row in df.iterrows():
    if row[1][2] == None or pd.isna(row[1][2]):
        temp_df.append(["None", 0, 0])
    else:
        try:
            items = row[1][2].split('(')
            sentiment = items[0].lower().strip()[0:-1]
            items = items[1].split(')')
            total = int(items[0].strip().replace(',', ''))
            percent = int(items[1].split('%')[0].strip()[3:]) / 100
            temp_df.append([sentiment, int(total * percent), total])
        except:
            temp_df.append(["None", 0, 0])
    idx += 1
    if idx % 5000 == 0:
        print(((idx + 1) / 40752) * 100, "%")
all_review_df = pd.DataFrame(temp_df, columns=all_review_cols)
df.drop(["all_reviews"], axis=1, inplace=True)
df = pd.concat([df, all_review_df], axis=1)
df['all_review_sentiment'] = df['all_review_sentiment'].astype('category').cat.codes
df['popular_tags'].fillna(df['genre'], inplace=True)
for i in range(len(df)):
    if pd.isna(df.loc[i, 'genre']):
        df.drop(i, inplace=True)
df.to_csv("steam_games_p2.csv", index=False)

12.271790341578328 %
24.541126815861798 %
36.810463290145265 %
49.07979976442874 %
61.34913623871221 %
73.61847271299568 %
85.88780918727915 %
98.15714566156262 %


In [15]:
df

,url,name,popular_tags,game_details,languages,genre,game_description,minimum_requirements,recommended_requirements,original_price,all_review_sentiment,all_review_sentiment_num,all_review_num
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,"FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",Action,"About This Game Developed by id software, the...","Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,9,39146,42550
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,"Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...","Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,1,409937,836608
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,"Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian","Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,3,4991,7030
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,"Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...","Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,1,101940,167115
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,"Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French","Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,3,8495,11481
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40604,https://store.steampowered.com/app/899836/Rock...,Rocksmith® 2014 Edition – Remastered – Sabaton...,"Casual,Simulation","Single-player,Shared/Split Screen,Downloadable...","English,German,French,Italian,Spanish - Spain,...","Casual,Simulation","About This Content Play ""Ghost Division"" by S...","Minimum:,OS:,Windows Vista, Windows 7, Windows...","Recommended:,OS:,Windows Vista, Windows 7, Win...",$2.99,0,0,0
40605,https://store.steampowered.com/app/899832/Rock...,Rocksmith® 2014 Edition – Remastered – Stone T...,"Casual,Simulation","Single-player,Shared/Split Screen,Downloadable...","English,German,French,Italian,Spanish - Spain,...","Casual,Simulation","About This Content Play ""Trippin’ on a Hole i...","Minimum:,OS:,Windows Vista, Windows 7, Windows...","Recommended:,OS:,Windows Vista, Windows 7, Win...",$2.99,0,0,0
40606,https://store.steampowered.com/app/906840/Fant...,Fantasy Grounds - Quests of Doom 4: A Midnight...,"RPG,Indie,Strategy,Software,Turn-Based,Fantasy...","Multi-player,Co-op,Cross-Platform Multiplayer,...",English,"Indie,RPG,Strategy",About This Content Quests of Doom 4: A Midni...,"Minimum:,OS:,Windows 7x , 8x or 10x,Processor:...","Recommended:,OS:,Windows 7x , 8x or 10x,Proces...",$7.99,0,0,0
40607,https://store.steampowered.com/app/906635/Mega...,Mega Man X5 Sound Collection,Action,"Single-player,Downloadable Content,Steam Achie...","English,French,Italian,German,Spanish - Spain,...",Action,About This Content Get equipped with the stun...,"Minimum:,OS:,WINDOWS® 7 (64bit),Processor:,Int...","Recommended:,OS:,WINDOWS®10 (64bit),Processor:...",$9.99,0,0,0


In [12]:
df = pd.read_csv('steam_games_p2.csv')

popular_tags = set()
pt_list = []

tag_list = df['popular_tags'].to_list()
genre_list = df['genre'].to_list()
for i in range(len(tag_list)):
    temp = tag_list[i].split(',')
    temp.extend(genre_list[i].split(','))
    temp = list(set(temp))
    pt_list.extend(temp)
    for j in temp:
        popular_tags.add(j.lower().strip())
pd.Series(pt_list).to_csv('popular_tags_freq.csv', index=False)

In [10]:
popular_tags = list(popular_tags)
tag_df_cols = ["url", "name"]
tag_df_cols.extend(popular_tags)

idx = 0
tag_map = {}
for tag in popular_tags:
    tag_map[tag] = idx + 2
    idx += 1
    
idx = 0
df = pd.read_csv('steam_games_p2.csv')
n = len(popular_tags)
temp_df = []
for row in df.iterrows():
    items = [row[1][0], row[1][1]]
    items.extend([0] * n)
    for tag in popular_tags:
        if row[1][2].find(tag) > -1:
            items[tag_map[tag]] = 1
        if row[1][5].find(tag) > -1:
            items[tag_map[tag]] = 1
    # tag_df.loc[idx] = items
    temp_df.append(items)
    idx += 1
    if idx % 5000 == 0:
        print(((idx + 1) / 40752) * 100, "%")
df.drop(["popular_tags", "genre"], axis=1, inplace=True)
tag_df = pd.DataFrame(temp_df, columns=tag_df_cols)
tag_df.to_csv("steam_games_tags.csv", index=False)

12.271790341578328 %
24.541126815861798 %
36.810463290145265 %
49.07979976442874 %
61.34913623871221 %
73.61847271299568 %
85.88780918727915 %
98.15714566156262 %


In [11]:
df

,url,name,game_details,languages,game_description,minimum_requirements,recommended_requirements,original_price,all_review_sentiment,all_review_sentiment_num,all_review_num
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,"Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...","About This Game Developed by id software, the...","Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,9,39146,42550
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,"Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,1,409937,836608
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,"Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",About This Game From original BATTLETECH/Mec...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,3,4991,7030
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,"Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",About This Game The post-soviet country of Ch...,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,1,101940,167115
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,"Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",About This Game,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,3,8495,11481
...,...,...,...,...,...,...,...,...,...,...,...
40323,https://store.steampowered.com/app/899836/Rock...,Rocksmith® 2014 Edition – Remastered – Sabaton...,"Single-player,Shared/Split Screen,Downloadable...","English,German,French,Italian,Spanish - Spain,...","About This Content Play ""Ghost Division"" by S...","Minimum:,OS:,Windows Vista, Windows 7, Windows...","Recommended:,OS:,Windows Vista, Windows 7, Win...",$2.99,0,0,0
40324,https://store.steampowered.com/app/899832/Rock...,Rocksmith® 2014 Edition – Remastered – Stone T...,"Single-player,Shared/Split Screen,Downloadable...","English,German,French,Italian,Spanish - Spain,...","About This Content Play ""Trippin’ on a Hole i...","Minimum:,OS:,Windows Vista, Windows 7, Windows...","Recommended:,OS:,Windows Vista, Windows 7, Win...",$2.99,0,0,0
40325,https://store.steampowered.com/app/906840/Fant...,Fantasy Grounds - Quests of Doom 4: A Midnight...,"Multi-player,Co-op,Cross-Platform Multiplayer,...",English,About This Content Quests of Doom 4: A Midni...,"Minimum:,OS:,Windows 7x , 8x or 10x,Processor:...","Recommended:,OS:,Windows 7x , 8x or 10x,Proces...",$7.99,0,0,0
40326,https://store.steampowered.com/app/906635/Mega...,Mega Man X5 Sound Collection,"Single-player,Downloadable Content,Steam Achie...","English,French,Italian,German,Spanish - Spain,...",About This Content Get equipped with the stun...,"Minimum:,OS:,WINDOWS® 7 (64bit),Processor:,Int...","Recommended:,OS:,WINDOWS®10 (64bit),Processor:...",$9.99,0,0,0
